### Read in the metadata files.

In [2]:
import numpy as np
import string
import pandas as pd
import sys
import re


with open('w09-data.out') as f:
    lines = f.readlines()
    arcs = np.array(list(map(lambda x : x.strip().split(), lines[1:11])))
    reads = np.array(list(map(lambda x : x.strip().split(), lines[13:23])))

arc_composition = arcs[:,3]
arc_lengths = [len(arc) for arc in arc_composition]
arc_names = arcs[:,0]
read_names = reads[:,0]
supplement_counts = reads[:,1].astype(int)
supplement_counts = pd.Series(supplement_counts)
supplement_counts.index = read_names



## 1. Write a simulator as a positive control

In [3]:
def read_simulator(arc_loc_sturcture,tau):
    
    # intialize list of generated reads
    all_reads = []
    length = [len(arc) for arc in arc_loc_sturcture]
    # convert tau to nu
    nu = (np.array(length) * np.array(tau)) / (np.sum(np.array(length) * np.array(tau)))
    # Choose a transcript isoform i with probability ν_i, according to the nucleotide abundances; i.e. P(T)=ν.
    isoform = np.random.choice(arc_loc_sturcture,size = 1000000, p=nu)
    for i in range(0,1000000):
        
        #Given isoform i, we choose one of the segments j=0..Li−1 of Ti uniformly -- i.e. 
        # for isoform Ti of length Li, we sample its segments with probability 1/Li.
        all_reads.append(np.random.choice(list(isoform[i].lower())))

    return all_reads

In [4]:
np.random.seed(42)
test_tau = np.random.dirichlet(np.ones(10),size=1).ravel()
all_reads = read_simulator(arc_composition,test_tau)
print('First 20 generated reads:')
all_reads[1:20]

First 20 generated reads:


['b',
 'i',
 'b',
 'c',
 'b',
 'd',
 'd',
 'd',
 'd',
 'h',
 'c',
 'c',
 'e',
 'f',
 'i',
 'c',
 'f',
 'e',
 'd']

### I first sampled probability vector $\tau$ of size 10 as my test case for $\tau$, where the probability vector $\tau$ sum up to one. This set of $\tau$ is then passed into my simulator function to generate a positive control

## 2. Calculate the log likelihhod

In [5]:
from scipy.special import logsumexp

def log_likelihood(r_count,arc_loc_sturcture,tau=None,nu=None):
    # Get length of each transcript
    length = np.array([len(arc) for arc in arc_loc_sturcture])
    # convert transcript abundance to nucletoide abundance
    if type(tau) == np.ndarray:
        nu = (np.array(length) * np.array(tau)) / (np.sum(np.array(length) * np.array(tau)))
    prob_s_given_g_all = 1 / length 
    
    # this is a variable tracking the cumlative summation of log likelihoods
    total_log_cum = 0

    # Iterate over each read
    for read in r_count.index:
        # This variable stores the log joint-probabilities of P(S,T ∣ ν,L) for the 10 genes given a particiular read (a...j)
        log_prob_joint = []
        # Iterate through the 10 genes, if a read is in a gene then P(S,T ∣ ν,L) is calculated and stored in log_prob_joint
        for gene,abund in enumerate(nu):
            if read in arc_loc_sturcture[gene].lower():
                prob_s_given_g = prob_s_given_g_all[gene]
                log_prob_joint.append((np.log(prob_s_given_g) + np.log(abund)))

        # Sum up log(P(S∣ν,L)) multiplied by number of times this read appears in the read counts.
        total_log_cum +=  logsumexp(log_prob_joint) * r_count[read]
           
             
    return total_log_cum

### The above algorithm of calculating the log-likelihood given read counts and $\tau$ is described as the following steps:
### First the passed in $\tau$ is converted to $\nu$. Then we calculate P(S∣T,L) through 1/L. Next we initalize a variable called total_log_cum that stores the cumlative summation of log likelihoods. We then enter into the main algorithm ---- a nested for loop structure. In the outer loop, we iterate over each read (a..j), we then initialize a intermediate variable called log_prob_joint to store the log joint-probabilities P(S,T ∣ ν,L) for the 10 genes given a particiular read (a...j). We then loop through each gene, and if the read exists in the gene, we calculate P(S,T ∣ ν,L) to store into log_prob_joint. After each inner for loop is terminated, we marginalize P(S,T | v, L) to get log(P(S∣ν,L)) and multiply it to the number of counts for that particular read and add the result to the cumlative summation of log likelihoods. The resulting cumlative summation of log likelihood after the nested for loop is termintaed is the total log likelihood we are looking for.

In [6]:
test = log_likelihood(pd.Series(all_reads).value_counts().sort_index(),arc_composition,tau=test_tau)
print('Log-likelihood of my generated test case')
test

Log-likelihood of my generated test case


-2217763.8797335

### Log Likelihood from Lestrade's approach

In [7]:
with open('w09-data.out') as f:
    #   The first line is "The <n> transcripts of the sand mouse Arc locus"
    line  = f.readline()
    match = re.search(r'^The (\d+) transcripts', line)
    T     = int(match.group(1))

    # The next T lines are 
    #   <Arcn>  <true_tau> <L> <structure>
    # tau's may be present, or obscured ("xxxxx")
    tau       = np.zeros(T)
    L         = np.zeros(T).astype(int)
    tau_known = True   # until we see otherwise
    for i in range(T):
        fields    = f.readline().split()
        if fields[1] == "xxxxx":
            tau_known = False
        else:
            tau[i] = float(fields[1])
        L[i]      = int(fields[2])

    # after a blank line,
    # 'The <n> read sequences':
    line  = f.readline()
    line  = f.readline()
    match = re.search(r'The (\d+) read sequences', line)
    N     = int(match.group(1))

    # the next T lines are 
    #  <read a-j> <count>


r = pd.Series(all_reads).value_counts().sort_index().ravel()


S = T    # S = R = T : there are T transcripts (Arc1..Arc10), S segments (A..J), R reads (a..j)
R = T
Slabel   = list(string.ascii_uppercase)[:S]               # ['A'..'J']        : the upper case labels for Arc locus segments 
Tlabel   = [ "Arc{}".format(d) for d in range(1,T+1) ]    # ['Arc1'..'Arc10'] : the labels for Arc transcript isoforms
Rlabel   = list(string.ascii_lowercase)[:T]               # ['a'..'j']        : lower case labels for reads


# Count how often each segment A..J is used in the isoforms i
# We'll use that to split observed read counts across the isoforms
# that they might have come from.
#
segusage = np.zeros(S).astype(int)
for i in range(T):
    for j in range(i,i+L[i]): 
        segusage[j%S] += 1


# Naive analysis:
#
c  = np.zeros(T)
for i in range(T):
    for k in range(i,i+L[i]):
        c[i] += (1.0 / float(segusage[k%S])) * float(r[k%S])  # For each read k, assume read k-> segment j,
                                                              # and assign 1/usage count to each transcript
                                                              # that contains segment j.
Z       = np.sum(c)
est_nu  = np.divide(c, Z)       # nucleotide abundance

print(c)

est_tau = np.divide(est_nu, L)  # convert to TPM, transcript abundance
est_tau = np.divide(est_tau, np.sum(est_tau))

# Print a table of the resulting estimates for tau
for i in range(T):
    print ("{0:10s}  {1:5.3f}".format(Tlabel[i], est_tau[i]))

[181268.66666667 116255.33333333 120803.          89508.25
  78638.25        49317.58333333  34077.25        75900.
 125137.33333333 129094.33333333]
Arc1        0.133
Arc2        0.170
Arc3        0.118
Arc4        0.065
Arc5        0.057
Arc6        0.048
Arc7        0.050
Arc8        0.111
Arc9        0.122
Arc10       0.126


In [8]:
lestrade_log_likelihood = log_likelihood(pd.Series(all_reads).value_counts().sort_index(),arc_composition,est_tau)
print("Lestrade's Tau:")
print(lestrade_log_likelihood)
print("Difference in log-likelihood: truth vs Lestrade's Tau")
print(test - lestrade_log_likelihood)

Lestrade's Tau:
-2240807.3180777426
Difference in log-likelihood: truth vs Lestrade's Tau
23043.438344242517


### We observe that the log likelihood from the test-case true $\tau$ is greater than the log likelihood from Lestrade's estimate $\tau$. 

## 3. estimate isoform abundances by EM

In [9]:
import pandas as pd

def expectation(r_count,arc_loc_sturcture,prob_s_given_g_all,nu):


    posterior_numerator =  pd.DataFrame(columns = arc_names,index = r_count.index)
    
    for read in r_count.index:
        # Iterate through the 10 genes, if a read is in a gene then P(S,T ∣ ν,L) is calculated and stored in log_prob_joint
        for gene,abund in enumerate(nu):
            if read in arc_loc_sturcture[gene].lower():
                prob_s_given_g = prob_s_given_g_all[gene]
                posterior_numerator.loc[read,arc_names[gene]] = (prob_s_given_g * abund)
    
    # obtaining the joint posterior probabilities by dividing each joint probability by the 
    # marginalized probability over each gene
    
    joint_posterior = posterior_numerator.div(posterior_numerator.sum(axis=1), axis=0)
    # expected count given posterior probaiblities P(v,|S,L)
    expected_count = joint_posterior.multiply(r_count,axis=0).sum(axis=0)

    return expected_count


def Maximization(expected_count,r_count):
    updated_nu = expected_count / r_count.sum()
    return updated_nu

def em_algorithm(r_count,arc_loc_sturcture,initial_nu):
    
    # intialize random nus

    prev_log_likelihood = log_likelihood(r_count,arc_loc_sturcture,nu=initial_nu)

    length = np.array([len(arc) for arc in arc_loc_sturcture])
    # convert transcript abundance to nucletoide abundance
    prob_s_given_g_all = 1 / length 
    
    new_log_likelihood = 10000000
    iter = 0
    while np.abs(new_log_likelihood - prev_log_likelihood) > 0.01:
        prev_log_likelihood = new_log_likelihood
        
        if iter == 0:
            expected_count = expectation(r_count,arc_loc_sturcture,prob_s_given_g_all,nu = initial_nu)
        else:
            expected_count = expectation(r_count,arc_loc_sturcture,prob_s_given_g_all,nu = updated_nu)

        updated_nu = Maximization(expected_count,r_count)
     
        new_log_likelihood = log_likelihood(r_count,arc_loc_sturcture,nu=updated_nu)
        iter+=1

    tau = (updated_nu / length) / (np.sum((updated_nu / length)))
    return tau, new_log_likelihood

np.random.seed(42)
# Running EM with 20 different sets of random nus
all_estimated_tau = []
all_log_likelihood = []
for i in range(0,20):
  
    random_nu = np.random.dirichlet(np.ones(10),size=1).ravel()
    my_estimated_tau, my_log_likelihood = em_algorithm(supplement_counts,arc_composition,initial_nu = random_nu)
    all_estimated_tau.append(my_estimated_tau)
    all_log_likelihood.append(my_log_likelihood)

best_run = np.argmax(all_log_likelihood)
my_estimated_tau = all_estimated_tau[best_run]
print('My estimated Tau to the read counts in Lestrade et al. supplementary data file:')
print(my_estimated_tau)
print('Sum of Arc1 and Arc2 abundances:')
my_estimated_tau[0] + my_estimated_tau[1]

My estimated Tau to the read counts in Lestrade et al. supplementary data file:
Arc1     0.148088
Arc2     0.497867
Arc3     0.001895
Arc4     0.048769
Arc5     0.037587
Arc6     0.019331
Arc7     0.006772
Arc8     0.134462
Arc9     0.054924
Arc10    0.050305
dtype: object
Sum of Arc1 and Arc2 abundances:


0.645954862736237

### I ran the EM algorithm with 20 different seed and obtained the $\tau$ estimates that yielded the highest log-likelihood. We observe the most abundant two transcripts is Arc 2 and Arc 1. They account for around 65% of the population. The two least abundant transcripts are Arc 3 and Arc 7. I do not agree with Lestrade et al.'s conclusion that both Arc 2 and Arc 3 are strongly on. Arc 2 has the most abundance and Arc 3 has the least abundance. Thus, this seems to support the original belief that Arc 2 and Arc 3 are oppositely regulated. Lestrade's method is wrong because Lestrade assumed read counts are assigned to possible isoforms uniformly. This is not an assumption for the EM algorithm